# Synthea Covid-19 Preprocessing

## Synthea COVID-19 Module Analysis

This notebook provides and analysis of data generated by Synthea's COVID-19 module. Analysis is run on the CSV output from Synthea.

Code in this notebook depends on Pandas, NumPy, matplotlib and seaborn.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import datetime

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 1
%aimport analysis

Read in all of the data

In [ ]:
data_path = "./raw_data/"
conditions = pd.read_csv(data_path + "conditions.csv")
patients = pd.read_csv(data_path + "patients.csv")
observations = pd.read_csv(data_path + "observations.csv")
care_plans = pd.read_csv(data_path + "careplans.csv")
encounters = pd.read_csv(data_path + "encounters.csv")
devices = pd.read_csv(data_path + "devices.csv")
supplies = pd.read_csv(data_path + 'supplies.csv')
procedures = pd.read_csv(data_path + "procedures.csv")
medications = pd.read_csv(data_path + "medications.csv")

Grab the IDs of patients that have been diagnosed with COVID-19

In [ ]:
covid_patient_ids = conditions[conditions.CODE == 840539006].PATIENT.unique()
covid_patient_ids[0:3]

This grabs every patient with a negative SARS-CoV-2 test. This will include patients who tested negative up front as well as patients that tested negative after leaving the hospital

In [ ]:
negative_covid_patient_ids = observations[(observations.CODE == '94531-1') & (observations.VALUE == 'Not detected (qualifier value)')].PATIENT.unique()
negative_covid_patient_ids[0:3]

Grabs IDs for all patients that died in the simulation. This will be more than just COVID-19 deaths.

In [ ]:
deceased_patients = patients[patients.DEATHDATE.notna()].Id

Grabs IDs for patients that have completed the care plan for isolation at home.

In [ ]:
completed_isolation_patients = care_plans[(care_plans.CODE == 736376001) & (care_plans.STOP.notna()) & (care_plans.REASONCODE == 840539006)].PATIENT

Survivors are the union of those who have completed isolation at home or have a negative SARS-CoV-2 test.

In [ ]:
survivor_ids = np.union1d(completed_isolation_patients, negative_covid_patient_ids)

Grab IDs for patients with admission due to COVID-19

In [ ]:
inpatient_ids = encounters[(encounters.REASONCODE == 840539006) & (encounters.CODE == 1505002)].PATIENT

The number of inpatient survivors

In [ ]:
np.intersect1d(inpatient_ids, survivor_ids).shape

The number of inpatient non-survivors

In [ ]:
np.intersect1d(inpatient_ids, deceased_patients).shape

In [ ]:
inpatient_ids.shape

## Health outcomes experienced by COVID-19 patients

The following table shows different health outcomes that were experienced by COVID-19 patients during the course of the disease.

In [ ]:
analysis.outcome_table(inpatient_ids, survivor_ids, deceased_patients, conditions)

## Lab values for COVID-19 patients

The following code presents lab values taken for COVID-19 patients. Values are separated into survivors and non survivors.

The first block of code selects lab values of interest from all observations in the simulation.

In [ ]:
# lab_obs = observations[(observations.CODE == '48065-7') | (observations.CODE == '26881-3') | 
#                           (observations.CODE == '2276-4') | (observations.CODE == '89579-7') |
#                           (observations.CODE == '2532-0') | (observations.CODE == '731-0') |
#                           (observations.CODE == '14804-9')
#                       ] # TODO more lab tests values
labtest_features = observations.CODE.unique().tolist()
# lab_obs[0:3]
# labtest_features[0:3]
lab_obs = observations[observations.CODE.isin(labtest_features)] # TODO

Select COVID-19 conditions out of all conditions in the simulation

In [ ]:
covid_conditions = conditions[conditions.CODE == 840539006]
covid_conditions[0:3]

Merge the COVID-19 conditions with the patients

In [ ]:
covid_patients = covid_conditions.merge(patients, how='left', left_on='PATIENT', right_on='Id')
covid_patients[0:3]

Add an attribute to the DataFrame indicating whether this is a survivor or not.

In [ ]:
covid_patients['survivor'] = covid_patients.PATIENT.isin(survivor_ids)
covid_patients

Reduce the columns on the DataFrame to ones needed

In [ ]:
covid_patients = covid_patients[['PATIENT', 'START', 'STOP', 'BIRTHDATE', 'DEATHDATE', 'survivor', 'CODE', 'GENDER']] # TODO other columns to consider
covid_patients[0:3]

Calculate attributes needed to support the plot. Also coerce all lab values into a numeric data type.

In [ ]:
covid_patients_obs = covid_patients.merge(lab_obs, on='PATIENT')
covid_patients_obs['START'] = pd.to_datetime(covid_patients_obs.START)
covid_patients_obs['STOP'] = pd.to_datetime(covid_patients_obs.STOP)
covid_patients_obs['BIRTHDATE'] = pd.to_datetime(covid_patients_obs.BIRTHDATE)
covid_patients_obs['DATE'] = pd.to_datetime(covid_patients_obs.DATE)
# covid_patients_obs['lab_days'] = covid_patients_obs.DATE - covid_patients_obs.START
# covid_patients_obs['days'] = covid_patients_obs.lab_days / np.timedelta64(1, 'D')
# covid_patients_obs['los_days'] = covid_patients_obs.STOP - covid_patients_obs.DATE
covid_patients_obs['LOS'] = (covid_patients_obs.STOP - covid_patients_obs.DATE) / np.timedelta64(1, 'D')
covid_patients_obs['VALUE'] = pd.to_numeric(covid_patients_obs['VALUE'], errors='coerce')
# covid_patients_obs['VALUE'] = covid_patients_obs['VALUE']
covid_patients_obs['AGE'] = (covid_patients_obs.DATE - covid_patients_obs.BIRTHDATE) / np.timedelta64(1, 'Y')

covid_patients_obs
# covid_patients_obs['days']